In [44]:
# Mengimpor library yang diperlukan
import numpy as np
import pandas as pd
import torch
import torch.utils.data
from torch import Tensor as tt

In [45]:
# Menyiapkan training set dan test set
training_set = pd.read_csv("https://raw.githubusercontent.com/azzandwi1/recommender-system-using-RBM/main/ml-100k/u1.base", sep='\t', header=None)
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [46]:
test_set = pd.read_csv("https://raw.githubusercontent.com/azzandwi1/recommender-system-using-RBM/main/ml-100k/u1.test", sep='\t', header=None)
test_set.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [47]:
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

In [48]:
# Menentukan jumlah maksimum pengguna (users) dan film untuk dibuat matriks
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [49]:
# Merubah data menjadi sebuah array (matriks) di mana baris adalah users dan kolom adalah film
def konversi_matriks(data):
    data_baru = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        data_baru.append(list(ratings))
    return data_baru

In [50]:
# Mengaplikasikan fungsi konversi_matriks ke training set dan test set
training_set = konversi_matriks(training_set)
test_set = konversi_matriks(test_set)

In [51]:
# Mengubah training_set dan test_set ke dalam Torch Tensor
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [52]:
# Merubah rating menjadi binary = 1 (suka) dan 0 (tidak suka)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [53]:
# Membuat rancangan arsitektur neural networks RBM
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_untuk_v = torch.sigmoid(activation)
        return p_h_untuk_v, torch.bernoulli(p_h_untuk_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_untuk_h = torch.sigmoid(activation)
        return p_v_untuk_h, torch.bernoulli(p_v_untuk_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
    def predict(self, x):
        _,h = self.sample_h(x)
        _,v = self.sample_v(h)
        return v

In [54]:
# Menentukan jumlah nv, nh dan batch_size
nv = 1682
nh = 100
batch_size = 100
rbm = RBM(nv, nh)
nb_epoch = 10

In [55]:
# Proses training RBM
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users + 1, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3405)
epoch: 2 loss: tensor(0.2420)
epoch: 3 loss: tensor(0.2448)
epoch: 4 loss: tensor(0.2475)
epoch: 5 loss: tensor(0.2439)
epoch: 6 loss: tensor(0.2472)
epoch: 7 loss: tensor(0.2461)
epoch: 8 loss: tensor(0.2459)
epoch: 9 loss: tensor(0.2464)
epoch: 10 loss: tensor(0.2472)


In [56]:
# Proses testing RBM
test_loss = 0
s = 0.
for id_user in range(nb_users + 1):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2070)


In [57]:
# Mengambil contoh id_user untuk diprediksi perilakunya
user_id = 0
while (user_id <= 0 or user_id >= 944):
    user_id = int(input('masukkan user_id yang diinginkan (antara 1-943) = '))
    if (user_id <= 0 or user_id >= 944):
        print('Masukkan angka antara 1-943')

user_input_training = tt(training_set[user_id-1]).unsqueeze(0)
user_input_test = tt(test_set[user_id-1]).unsqueeze(0)

# %%
# Membuat prediksi untuk training set
prediksi_training = rbm.predict(user_input_training)
prediksi_training = prediksi_training.data.numpy()
prediksi_film_training = np.vstack([user_input_training, prediksi_training])

# Menampilkan hasil prediksi untuk training set
print("Prediksi untuk training set:")
print(prediksi_film_training)

# %%
# Membuat prediksi untuk test set
prediksi_test = rbm.predict(user_input_test)
prediksi_test = prediksi_test.data.numpy()
prediksi_film_test = np.vstack([user_input_test, prediksi_test])

# Menampilkan hasil prediksi untuk test set
print("Prediksi untuk test set:")
print(prediksi_film_test)

Prediksi untuk training set:
[[-1. -1. -1. ... -1. -1. -1.]
 [ 1.  1.  1. ...  0.  1.  1.]]
Prediksi untuk test set:
[[-1. -1. -1. ... -1. -1. -1.]
 [ 1.  1.  1. ...  0.  0.  1.]]
